In [2]:
from gurobipy import GRB, Model, quicksum
import gurobipy as gb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker


In [3]:
workout = pd.read_csv('C:/Users/etiem/Downloads/updated_gym_data.csv')

In [4]:
workout

,Exercise,Category,BodyPart,Equipment,Difficulty,Stimulus-to-Fatigue,Expected Time,Hypertrophy Rating
0,Bench Press With Short Bands,Powerlifting,Chest,Bands,Beginner,0.817884,15.518089,0.596124
1,Hip Lift with Band,Powerlifting,Glutes,Bands,Beginner,0.768902,14.655351,0.623237
2,Band Good Morning (Pull Through),Powerlifting,Hamstrings,Bands,Beginner,0.792188,16.292358,0.601159
3,Speed Box Squat,Powerlifting,Quadriceps,Bands,Intermediate,0.599044,17.109781,0.800347
4,Partner plank band row,Strength,Abdominals,Bands,Intermediate,0.730726,14.212727,0.461565
...,...,...,...,...,...,...,...,...
2632,Sandbag Load,Strongman,Quadriceps,Other,Beginner,0.364982,23.073234,0.519070
2633,Car Deadlift,Strongman,Quadriceps,Other,Beginner,0.465682,22.598407,0.599859
2634,Circus Bell,Strongman,Shoulders,Other,Beginner,0.425416,21.003540,0.609114
2635,Crucifix,Strongman,Shoulders,Other,Beginner,0.419535,22.624371,0.633115


In [5]:
print(workout['Exercise'].unique())


['Bench Press With Short Bands' 'Hip Lift with Band'
 'Band Good Morning (Pull Through)' ... 'Circus Bell' 'Crucifix'
 'Log Lift']


# Note 
Please note that constraint 4 was turned into 2 constraints for ease of editing the code, therefore the code states there are 9 constraints even though the assignment states there are 8 constraints.

In [6]:


# %%
# --------------------------
# Reclassify Back Subparts
# --------------------------
back_subparts = ["Lats", "Lower Back", "Middle Back"]  # Adjust based on your dataset
workout.loc[workout['BodyPart'].isin(back_subparts), 'BodyPart'] = "Back"

# --------------------------
# Filter High-SFR Chest Exercises
# --------------------------
workout = workout[
    ~((workout['BodyPart'] == "Chest") & (workout['Stimulus-to-Fatigue'] > 0.55))
]

# Reset indices to avoid KeyError
workout = workout.reset_index(drop=True)

# --------------------------
# Define the Model and Constraints
# --------------------------
model = Model("Workout_Optimization")
x = model.addVars(len(workout), lb=0, ub=0.05, vtype=GRB.CONTINUOUS, name="x")

# Objective: Maximize hypertrophy rating
hypertrophy = workout['Hypertrophy Rating'].values
model.setObjective(quicksum(hypertrophy[i] * x[i] for i in range(len(workout))), GRB.MAXIMIZE)

# Constraint 1: Total allocation = 1
model.addConstr(quicksum(x[i] for i in range(len(workout))) == 1, "Total_Allocation")

# Constraint 2: Body part minimums
body_parts_min = {
    "Traps": 0.005, "Neck": 0.005, "Forearms": 0.005, "Abdominals": 0.04
}
all_body_parts = workout['BodyPart'].unique()
for body_part in all_body_parts:
    exercises = workout[workout['BodyPart'] == body_part].index
    if body_part in body_parts_min:
        min_val = body_parts_min[body_part]
    else:
        min_val = 0.025  # 2.5% default
    model.addConstr(quicksum(x[i] for i in exercises) >= min_val, f"Min_{body_part}")

# Constraint 3: Leg muscles ≥ 2.6× Upper body
leg_muscles = ["Adductors", "Abductors", "Calves", "Glutes", "Hamstrings", "Quadriceps"]
leg_indices = workout[workout['BodyPart'].isin(leg_muscles)].index
upper_indices = workout[~workout['BodyPart'].isin(leg_muscles)].index
model.addConstr(
    quicksum(x[i] for i in leg_indices) >= 2.6 * quicksum(x[i] for i in upper_indices),
    name="Leg_vs_Upper"
)

# Constraint 4: Chest vs. Back equality
back_indices = workout[workout['BodyPart'] == "Back"].index  # Updated after reclassification
chest_indices = workout[workout['BodyPart'] == "Chest"].index
model.addConstr(
    quicksum(x[i] for i in chest_indices) == quicksum(x[i] for i in back_indices), 
    "Chest_Back"
)

# Constraint 5: Biceps vs Triceps equality
biceps_indices = workout[workout['BodyPart'] == "Biceps"].index
triceps_indices = workout[workout['BodyPart'] == "Triceps"].index
model.addConstr(
    quicksum(x[i] for i in biceps_indices) == quicksum(x[i] for i in triceps_indices),
    "Biceps_Triceps"
)

# Constraint 6: SFR ≤ 0.55
sfr = workout['Stimulus-to-Fatigue'].values
model.addConstr(quicksum(sfr[i] * x[i] for i in range(len(workout))) <= 0.55, name="SFR")

# Constraint 7: Beginner/Intermediate/Advanced ratios
beginner_indices = workout[workout['Difficulty'] == "Beginner"].index
intermediate_indices = workout[workout['Difficulty'] == "Intermediate"].index
expert_indices = workout[workout['Difficulty'] == "Expert"].index
model.addConstr(quicksum(x[i] for i in beginner_indices) >= 1.4 * quicksum(x[i] for i in intermediate_indices))
model.addConstr(
    quicksum(x[i] for i in intermediate_indices) >= 1.1 * quicksum(x[i] for i in expert_indices), 
    "Intermediate_Expert"
)

# Constraint 8: Category constraints (Strongman, Powerlifting, Olympic)
strongman_indices = workout[workout['Category'] == "Strongman"].index
powerlifting_indices = workout[workout['Category'] == "Powerlifting"].index
olympic_indices = workout[workout['Category'] == "Olympic Weightlifting"].index

model.addConstr(quicksum(x[i] for i in strongman_indices) <= 0.08 - 1e-5, "Strongman_Limit")  # <8%
model.addConstr(quicksum(x[i] for i in powerlifting_indices) >= 0.09 + 1e-5, "Powerlifting_Min")  # >9%
model.addConstr(quicksum(x[i] for i in olympic_indices) >= 0.10 + 1e-5, "Olympic_Min")  # >10%

# Constraint 9: Equipment usage ≥60%
equipment_list = ["Barbell", "Dumbbell", "Machine", "Cable", "E-Z Curl Bar", "Bands"]
equipment_indices = workout[workout['Equipment'].isin(equipment_list)].index
model.addConstr(quicksum(x[i] for i in equipment_indices) >= 0.6, "Equipment_Min")

# Optimize the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Hypertrophy Rating: {model.objVal}")
    for i in range(len(workout)):
        if x[i].x > 0:
            print(f"{workout['Exercise'][i]}: {x[i].x:.4f}")
else:
    print("Model is infeasible. Check dataset or constraints!")

# %%
# Validation checks (same as before)
# Check available Chest/Back exercises
chest_exercises = workout[workout['BodyPart'] == "Chest"]
back_exercises = workout[workout['BodyPart'] == "Back"]
print(f"Chest exercises: {len(chest_exercises)}")
print(f"Back exercises: {len(back_exercises)}")

# Check SFR values for Chest/Back exercises
print("Chest SFR stats:", chest_exercises['Stimulus-to-Fatigue'].describe())
print("Back SFR stats:", back_exercises['Stimulus-to-Fatigue'].describe())

# %%
# Check categories and constraints
print("Categories:", workout['Category'].unique())
print("BodyParts:", workout['BodyPart'].unique())

# %%
# Check model status and IIS if needed
if model.status == GRB.INFEASIBLE:
    print("Model is infeasible. Computing IIS...")
    model.computeIIS()
    for c in model.getConstrs():
        if c.IISConstr:
            print(f"Conflict in constraint: {c.ConstrName}")

# %%
# Post-optimization validation
if model.status == GRB.OPTIMAL:
    # Check total allocation
    total = sum(x[i].x for i in range(len(workout)))
    print(f"Total Allocation: {total:.6f}")  # Should be 1.0

    # Check SFR
    sfr_total = sum(workout['Stimulus-to-Fatigue'][i] * x[i].x for i in range(len(workout)))
    print(f"SFR Total: {sfr_total:.6f}")  # Should be ≤0.55

    # Check Biceps/Triceps equality
    biceps_total = sum(x[i].x for i in biceps_indices)
    triceps_total = sum(x[i].x for i in triceps_indices)
    print(f"Biceps: {biceps_total:.4f}, Triceps: {triceps_total:.4f}")

    # Check category constraints
    strongman_total = sum(x[i].x for i in strongman_indices)
    powerlifting_total = sum(x[i].x for i in powerlifting_indices)
    olympic_total = sum(x[i].x for i in olympic_indices)
    print(f"Strongman: {strongman_total:.4f} (≤8%), Powerlifting: {powerlifting_total:.4f} (≥9%), Olympic: {olympic_total:.4f} (≥10%)")

Set parameter Username
Set parameter LicenseID to value 2609485


Academic license - for non-commercial use only - expires 2026-01-13
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 26 rows, 2400 columns and 15976 nonzeros
Model fingerprint: 0x77888ef9
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [5e-02, 5e-02]
  RHS range        [5e-03, 1e+00]
Presolve time: 0.02s
Presolved: 26 rows, 2400 columns, 15976 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   2.755010e+00   0.000000e+00      0s
      28    7.8574257e-01   0.000000e+00   0.000000e+00      0s

Solved in 28 iterations and 0.02 seconds (0.01 work units)
Optimal objective  7.857425731e-01
Optimal Hypertrophy Rating: 0.7857425730587836
Clean Deadlift: 0.0500
Muscle S

In [7]:
#This is just to double check that the strongman exercise is below 0.08 as it says in the constraint
strongman_total = sum(x[i].x for i in strongman_indices)
print(f"Exact Strongman Total: {strongman_total:.10f}")

Exact Strongman Total: 0.0799900000


In [8]:
# Check available Chest/Back exercises
chest_exercises = workout[workout['BodyPart'] == "Chest"]
back_exercises = workout[workout['BodyPart'] == "Back"]
print(f"Chest exercises: {len(chest_exercises)}")
print(f"Back exercises: {len(back_exercises)}")

# Check SFR values for Chest/Back exercises
print("Chest SFR stats:", chest_exercises['Stimulus-to-Fatigue'].describe())
print("Back SFR stats:", back_exercises['Stimulus-to-Fatigue'].describe())

Chest exercises: 6
Back exercises: 310
Chest SFR stats: count    6.000000
mean     0.528114
std      0.035189
min      0.456939
25%      0.536605
50%      0.541032
75%      0.544334
max      0.549116
Name: Stimulus-to-Fatigue, dtype: float64
Back SFR stats: count    310.000000
mean       0.671805
std        0.081490
min        0.256262
25%        0.617350
50%        0.664168
75%        0.716216
max        0.931359
Name: Stimulus-to-Fatigue, dtype: float64


In [9]:
#Here we are checking to ensure that all of our columns are included in the variables  
print("BodyParts:", workout['BodyPart'].unique())
print("Categories:", workout['Category'].unique())
print("Equipment:", workout['Equipment'].unique())
print("Difficulty:", workout['Difficulty'].unique())

BodyParts: ['Glutes' 'Hamstrings' 'Quadriceps' 'Abdominals' 'Adductors' 'Abductors'
 'Biceps' 'Calves' 'Forearms' 'Back' 'Traps' 'Shoulders' 'Triceps' 'Chest'
 'Neck']
Categories: ['Powerlifting' 'Strength' 'Olympic Weightlifting' 'Strongman']
Equipment: ['Bands' 'Barbell' 'Body Only' 'Cable' 'Dumbbell' 'Exercise Ball'
 'E-Z Curl Bar' 'Kettlebells' 'Machine' 'Medicine Ball' nan 'Other']
Difficulty: ['Beginner' 'Intermediate' 'Expert']


In [10]:
#This checks to see if the model is even working. Status 2 means it works, status 3 means its not
model.optimize()
print(f"Model Status: {model.status}")  

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 26 rows, 2400 columns and 15976 nonzeros
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [5e-02, 5e-02]
  RHS range        [5e-03, 1e+00]

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.857425731e-01
Model Status: 2


In [11]:
#If there is an error with the model this checks to see where the error in constraints occurs
if model.status == GRB.INFEASIBLE:
    print("Model is infeasible. Computing IIS to find conflicting constraints...")
    model.computeIIS()  # Identifies the Minimal Infeasible Subset (MIS)
    for c in model.getConstrs():
        if c.IISConstr:
            print(f"Conflict in constraint: {c.ConstrName}")

In [12]:
#For part C prints the optimal hypertrophy rating again just to make sure I have it distinct in the code
if model.status == GRB.OPTIMAL:
    print(f"Optimal Hypertrophy Rating: {model.objVal}")

Optimal Hypertrophy Rating: 0.7857425730587836


In [13]:
#Below we are running checks

# Check total allocation
total = sum(x[i].x for i in range(len(workout)))
print(f"Total Allocation: {total}")  # Should be 1.0

# Check SFR
sfr_total = sum(workout['Stimulus-to-Fatigue'][i] * x[i].x for i in range(len(workout)))
print(f"SFR Total: {sfr_total}")  # Should be ≤ 0.55

Total Allocation: 0.9999999999999998
SFR Total: 0.55


# Part D
### Where we relax Constraint 5

In [14]:


# %%
# --------------------------
# Reclassify Back Subparts
# --------------------------
back_subparts = ["Lats", "Lower Back", "Middle Back"]  # Adjust based on your dataset
workout.loc[workout['BodyPart'].isin(back_subparts), 'BodyPart'] = "Back"

# --------------------------
# Filter High-SFR Chest Exercises
# --------------------------
workout = workout[
    ~((workout['BodyPart'] == "Chest") & (workout['Stimulus-to-Fatigue'] > 0.55))
]

# Reset indices to avoid KeyError
workout = workout.reset_index(drop=True)

# --------------------------
# Define the Model and Constraints
# --------------------------
model = Model("Workout_Optimization")
x = model.addVars(len(workout), lb=0, ub=0.05, vtype=GRB.CONTINUOUS, name="x")

# Objective: Maximize hypertrophy rating
hypertrophy = workout['Hypertrophy Rating'].values
model.setObjective(quicksum(hypertrophy[i] * x[i] for i in range(len(workout))), GRB.MAXIMIZE)

# Constraint 1: Total allocation = 1
model.addConstr(quicksum(x[i] for i in range(len(workout))) == 1, "Total_Allocation")

# Constraint 2: Body part minimums
body_parts_min = {
    "Traps": 0.005, "Neck": 0.005, "Forearms": 0.005, "Abdominals": 0.04
}
all_body_parts = workout['BodyPart'].unique()
for body_part in all_body_parts:
    exercises = workout[workout['BodyPart'] == body_part].index
    if body_part in body_parts_min:
        min_val = body_parts_min[body_part]
    else:
        min_val = 0.025  # 2.5% default
    model.addConstr(quicksum(x[i] for i in exercises) >= min_val, f"Min_{body_part}")

# Constraint 3: Leg muscles ≥ 2.6× Upper body
leg_muscles = ["Adductors", "Abductors", "Calves", "Glutes", "Hamstrings", "Quadriceps"]
leg_indices = workout[workout['BodyPart'].isin(leg_muscles)].index
upper_indices = workout[~workout['BodyPart'].isin(leg_muscles)].index
model.addConstr(
    quicksum(x[i] for i in leg_indices) >= 2.6 * quicksum(x[i] for i in upper_indices),
    name="Leg_vs_Upper"
)

# Constraint 4: Chest vs. Back equality
back_indices = workout[workout['BodyPart'] == "Back"].index  # Updated after reclassification
chest_indices = workout[workout['BodyPart'] == "Chest"].index
model.addConstr(
    quicksum(x[i] for i in chest_indices) == quicksum(x[i] for i in back_indices), 
    "Chest_Back"
)

# Constraint 5: Biceps vs Triceps equality
biceps_indices = workout[workout['BodyPart'] == "Biceps"].index
triceps_indices = workout[workout['BodyPart'] == "Triceps"].index
model.addConstr(
    quicksum(x[i] for i in biceps_indices) == quicksum(x[i] for i in triceps_indices),
    "Biceps_Triceps"
)

# Constraint 6: SFR ≤ 0.55
sfr = workout['Stimulus-to-Fatigue'].values
model.addConstr(quicksum(sfr[i] * x[i] for i in range(len(workout))) <= 0.551, name="SFR")

# Constraint 7: Beginner/Intermediate/Advanced ratios
beginner_indices = workout[workout['Difficulty'] == "Beginner"].index
intermediate_indices = workout[workout['Difficulty'] == "Intermediate"].index
expert_indices = workout[workout['Difficulty'] == "Expert"].index
model.addConstr(quicksum(x[i] for i in beginner_indices) >= 1.4 * quicksum(x[i] for i in intermediate_indices))
model.addConstr(
    quicksum(x[i] for i in intermediate_indices) >= 1.1 * quicksum(x[i] for i in expert_indices), 
    "Intermediate_Expert"
)

# Constraint 8: Category constraints (Strongman, Powerlifting, Olympic)
strongman_indices = workout[workout['Category'] == "Strongman"].index
powerlifting_indices = workout[workout['Category'] == "Powerlifting"].index
olympic_indices = workout[workout['Category'] == "Olympic Weightlifting"].index

model.addConstr(quicksum(x[i] for i in strongman_indices) <= 0.08 - 1e-5, "Strongman_Limit")  # <8%
model.addConstr(quicksum(x[i] for i in powerlifting_indices) >= 0.09 + 1e-5, "Powerlifting_Min")  # >9%
model.addConstr(quicksum(x[i] for i in olympic_indices) >= 0.10 + 1e-5, "Olympic_Min")  # >10%

# Constraint 9: Equipment usage ≥60%
equipment_list = ["Barbell", "Dumbbell", "Machine", "Cable", "E-Z Curl Bar", "Bands"]
equipment_indices = workout[workout['Equipment'].isin(equipment_list)].index
model.addConstr(quicksum(x[i] for i in equipment_indices) >= 0.6, "Equipment_Min")

# Optimize the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Hypertrophy Rating: {model.objVal}")
    for i in range(len(workout)):
        if x[i].x > 0:
            print(f"{workout['Exercise'][i]}: {x[i].x:.4f}")
else:
    print("Model is infeasible. Check dataset or constraints!")

# %%
# Validation checks (same as before)
# Check available Chest/Back exercises
chest_exercises = workout[workout['BodyPart'] == "Chest"]
back_exercises = workout[workout['BodyPart'] == "Back"]
print(f"Chest exercises: {len(chest_exercises)}")
print(f"Back exercises: {len(back_exercises)}")

# Check SFR values for Chest/Back exercises
print("Chest SFR stats:", chest_exercises['Stimulus-to-Fatigue'].describe())
print("Back SFR stats:", back_exercises['Stimulus-to-Fatigue'].describe())

# %%
# Check categories and constraints
print("Categories:", workout['Category'].unique())
print("BodyParts:", workout['BodyPart'].unique())

# %%
# Check model status and IIS if needed
if model.status == GRB.INFEASIBLE:
    print("Model is infeasible. Computing IIS...")
    model.computeIIS()
    for c in model.getConstrs():
        if c.IISConstr:
            print(f"Conflict in constraint: {c.ConstrName}")

# %%
# Post-optimization validation
if model.status == GRB.OPTIMAL:
    # Check total allocation
    total = sum(x[i].x for i in range(len(workout)))
    print(f"Total Allocation: {total:.6f}")  # Should be 1.0

    # Check SFR
    sfr_total = sum(workout['Stimulus-to-Fatigue'][i] * x[i].x for i in range(len(workout)))
    print(f"SFR Total: {sfr_total:.6f}")  # Should be ≤0.55

    # Check Biceps/Triceps equality
    biceps_total = sum(x[i].x for i in biceps_indices)
    triceps_total = sum(x[i].x for i in triceps_indices)
    print(f"Biceps: {biceps_total:.4f}, Triceps: {triceps_total:.4f}")

    # Check category constraints
    strongman_total = sum(x[i].x for i in strongman_indices)
    powerlifting_total = sum(x[i].x for i in powerlifting_indices)
    olympic_total = sum(x[i].x for i in olympic_indices)
    print(f"Strongman: {strongman_total:.4f} (≤8%), Powerlifting: {powerlifting_total:.4f} (≥9%), Olympic: {olympic_total:.4f} (≥10%)")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 26 rows, 2400 columns and 15976 nonzeros
Model fingerprint: 0x9efef3cf
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [5e-02, 5e-02]
  RHS range        [5e-03, 1e+00]
Presolve time: 0.01s
Presolved: 26 rows, 2400 columns, 15976 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   2.755010e+00   0.000000e+00      0s
      26    7.8699268e-01   0.000000e+00   0.000000e+00      0s

Solved in 26 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.869926764e-01
Optimal Hypertrophy Rating: 0.7869926763775107
Clean Deadlift: 0.0500
Muscle Snatch: 0.0500
Clean Shrug: 0.0050
Clean from Blocks: 0.0337
Power sn

# Part E

In [15]:
#Part E



# %%
# --------------------------
# Reclassify Back Subparts
# --------------------------
back_subparts = ["Lats", "Lower Back", "Middle Back"]  # Adjust based on your dataset
workout.loc[workout['BodyPart'].isin(back_subparts), 'BodyPart'] = "Back"

# --------------------------
# Filter High-SFR Chest Exercises
# --------------------------
workout = workout[
    ~((workout['BodyPart'] == "Chest") & (workout['Stimulus-to-Fatigue'] > 0.55))
]

# Reset indices to avoid KeyError
workout = workout.reset_index(drop=True)

# --------------------------
# Define the Model and Constraints
# --------------------------
model = Model("Workout_Optimization")
x = model.addVars(len(workout), lb=0, ub=0.05, vtype=GRB.CONTINUOUS, name="x")

# Objective: Maximize hypertrophy rating
hypertrophy = workout['Hypertrophy Rating'].values
model.setObjective(quicksum(hypertrophy[i] * x[i] for i in range(len(workout))), GRB.MAXIMIZE)

# Constraint 1: Total allocation = 1
model.addConstr(quicksum(x[i] for i in range(len(workout))) == 1, "Total_Allocation")

# Constraint 2: Body part minimums
body_parts_min = {
    "Traps": 0.01,    # Increased from 0.005 (0.5% to 1%)
    "Neck": 0.01,     # Increased from 0.005
    "Forearms": 0.01, # Increased from 0.005
    "Abdominals": 0.05  # Increased from 0.04 (4% to 5%)
}
all_body_parts = workout['BodyPart'].unique()
for body_part in all_body_parts:
    exercises = workout[workout['BodyPart'] == body_part].index
    if body_part in body_parts_min:
        min_val = body_parts_min[body_part]
    else:
        min_val = 0.025  # 2.5% default
    model.addConstr(quicksum(x[i] for i in exercises) >= min_val, f"Min_{body_part}")

# Constraint 3: Leg muscles ≥ 2.6× Upper body
leg_muscles = ["Adductors", "Abductors", "Calves", "Glutes", "Hamstrings", "Quadriceps"]
leg_indices = workout[workout['BodyPart'].isin(leg_muscles)].index
upper_indices = workout[~workout['BodyPart'].isin(leg_muscles)].index
model.addConstr(
    quicksum(x[i] for i in leg_indices) >= 2.6 * quicksum(x[i] for i in upper_indices),
    name="Leg_vs_Upper"
)

# Constraint 4: Chest vs. Back equality
back_indices = workout[workout['BodyPart'] == "Back"].index  # Updated after reclassification
chest_indices = workout[workout['BodyPart'] == "Chest"].index
model.addConstr(
    quicksum(x[i] for i in chest_indices) == quicksum(x[i] for i in back_indices), 
    "Chest_Back"
)

# Constraint 5: Biceps vs Triceps equality
biceps_indices = workout[workout['BodyPart'] == "Biceps"].index
triceps_indices = workout[workout['BodyPart'] == "Triceps"].index
model.addConstr(
    quicksum(x[i] for i in biceps_indices) == quicksum(x[i] for i in triceps_indices),
    "Biceps_Triceps"
)

# Constraint 6: SFR ≤ 0.55
sfr = workout['Stimulus-to-Fatigue'].values
model.addConstr(quicksum(sfr[i] * x[i] for i in range(len(workout))) <= 0.55, name="SFR")

# Constraint 7: Beginner/Intermediate/Advanced ratios
beginner_indices = workout[workout['Difficulty'] == "Beginner"].index
intermediate_indices = workout[workout['Difficulty'] == "Intermediate"].index
expert_indices = workout[workout['Difficulty'] == "Expert"].index
model.addConstr(quicksum(x[i] for i in beginner_indices) >= 1.4 * quicksum(x[i] for i in intermediate_indices))
model.addConstr(
    quicksum(x[i] for i in intermediate_indices) >= 1.1 * quicksum(x[i] for i in expert_indices), 
    "Intermediate_Expert"
)

# Constraint 8: Category constraints (Strongman, Powerlifting, Olympic)
strongman_indices = workout[workout['Category'] == "Strongman"].index
powerlifting_indices = workout[workout['Category'] == "Powerlifting"].index
olympic_indices = workout[workout['Category'] == "Olympic Weightlifting"].index

model.addConstr(quicksum(x[i] for i in strongman_indices) <= 0.08 - 1e-5, "Strongman_Limit")  # <8%
model.addConstr(quicksum(x[i] for i in powerlifting_indices) >= 0.09 + 1e-5, "Powerlifting_Min")  # >9%
model.addConstr(quicksum(x[i] for i in olympic_indices) >= 0.10 + 1e-5, "Olympic_Min")  # >10%

# Constraint 9: Equipment usage ≥60%
equipment_list = ["Barbell", "Dumbbell", "Machine", "Cable", "E-Z Curl Bar", "Bands"]
equipment_indices = workout[workout['Equipment'].isin(equipment_list)].index
model.addConstr(quicksum(x[i] for i in equipment_indices) >= 0.6, "Equipment_Min")

# Optimize the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Hypertrophy Rating: {model.objVal}")
    for i in range(len(workout)):
        if x[i].x > 0:
            print(f"{workout['Exercise'][i]}: {x[i].x:.4f}")
else:
    print("Model is infeasible. Check dataset or constraints!")

# %%
# Validation checks (same as before)
# Check available Chest/Back exercises
chest_exercises = workout[workout['BodyPart'] == "Chest"]
back_exercises = workout[workout['BodyPart'] == "Back"]
print(f"Chest exercises: {len(chest_exercises)}")
print(f"Back exercises: {len(back_exercises)}")

# Check SFR values for Chest/Back exercises
print("Chest SFR stats:", chest_exercises['Stimulus-to-Fatigue'].describe())
print("Back SFR stats:", back_exercises['Stimulus-to-Fatigue'].describe())

# %%
# Check categories and constraints
print("Categories:", workout['Category'].unique())
print("BodyParts:", workout['BodyPart'].unique())

# %%
# Check model status and IIS if needed
if model.status == GRB.INFEASIBLE:
    print("Model is infeasible. Computing IIS...")
    model.computeIIS()
    for c in model.getConstrs():
        if c.IISConstr:
            print(f"Conflict in constraint: {c.ConstrName}")

# %%
# Post-optimization validation
if model.status == GRB.OPTIMAL:
    # Check total allocation
    total = sum(x[i].x for i in range(len(workout)))
    print(f"Total Allocation: {total:.6f}")  # Should be 1.0

    # Check SFR
    sfr_total = sum(workout['Stimulus-to-Fatigue'][i] * x[i].x for i in range(len(workout)))
    print(f"SFR Total: {sfr_total:.6f}")  # Should be ≤0.55

    # Check Biceps/Triceps equality
    biceps_total = sum(x[i].x for i in biceps_indices)
    triceps_total = sum(x[i].x for i in triceps_indices)
    print(f"Biceps: {biceps_total:.4f}, Triceps: {triceps_total:.4f}")

    # Check category constraints
    strongman_total = sum(x[i].x for i in strongman_indices)
    powerlifting_total = sum(x[i].x for i in powerlifting_indices)
    olympic_total = sum(x[i].x for i in olympic_indices)
    print(f"Strongman: {strongman_total:.4f} (≤8%), Powerlifting: {powerlifting_total:.4f} (≥9%), Olympic: {olympic_total:.4f} (≥10%)")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 26 rows, 2400 columns and 15976 nonzeros
Model fingerprint: 0x83de8d9f
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [5e-02, 5e-02]
  RHS range        [1e-02, 1e+00]
Presolve time: 0.01s
Presolved: 26 rows, 2400 columns, 15976 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   2.780010e+00   0.000000e+00      0s
      27    7.7976498e-01   0.000000e+00   0.000000e+00      0s

Solved in 27 iterations and 0.02 seconds (0.01 work units)
Optimal objective  7.797649780e-01
Optimal Hypertrophy Rating: 0.7797649779681055
Clean Deadlift: 0.0500
Muscle Snatch: 0.0500
Clean Shrug: 0.0100
Clean from Blocks: 0.0398
Power sn

# Part F

In [16]:


# %%
# --------------------------
# Reclassify Back Subparts
# --------------------------
back_subparts = ["Lats", "Lower Back", "Middle Back"]  # Adjust based on your dataset
workout.loc[workout['BodyPart'].isin(back_subparts), 'BodyPart'] = "Back"

# --------------------------
# Filter High-SFR Chest Exercises
# --------------------------
workout = workout[
    ~((workout['BodyPart'] == "Chest") & (workout['Stimulus-to-Fatigue'] > 0.55))
]

# Reset indices to avoid KeyError
workout = workout.reset_index(drop=True)

# --------------------------
# Define the Model and Constraints
# --------------------------
model = Model("Workout_Optimization")
x = model.addVars(len(workout), lb=0, ub=0.05, vtype=GRB.CONTINUOUS, name="x")

# Objective: Maximize hypertrophy rating
hypertrophy = workout['Hypertrophy Rating'].values
model.setObjective(quicksum(hypertrophy[i] * x[i] for i in range(len(workout))), GRB.MAXIMIZE)

# Constraint 1: Total allocation = 1
model.addConstr(quicksum(x[i] for i in range(len(workout))) == 1, "Total_Allocation")

# Constraint 2: Body part minimums
body_parts_min = {
    "Traps": 0.005, "Neck": 0.005, "Forearms": 0.005, "Abdominals": 0.04
}
all_body_parts = workout['BodyPart'].unique()
for body_part in all_body_parts:
    exercises = workout[workout['BodyPart'] == body_part].index
    if body_part in body_parts_min:
        min_val = body_parts_min[body_part]
    else:
        min_val = 0.025  # 2.5% default
    model.addConstr(quicksum(x[i] for i in exercises) >= min_val, f"Min_{body_part}")

# Constraint 3: Leg muscles ≥ 2.6× Upper body
leg_muscles = ["Adductors", "Abductors", "Calves", "Glutes", "Hamstrings", "Quadriceps"]
leg_indices = workout[workout['BodyPart'].isin(leg_muscles)].index
upper_indices = workout[~workout['BodyPart'].isin(leg_muscles)].index
model.addConstr(
    quicksum(x[i] for i in leg_indices) >= 2.6 * quicksum(x[i] for i in upper_indices),
    name="Leg_vs_Upper"
)

# Constraint 4: Chest vs. Back equality
back_indices = workout[workout['BodyPart'] == "Back"].index  # Updated after reclassification
chest_indices = workout[workout['BodyPart'] == "Chest"].index
model.addConstr(
    quicksum(x[i] for i in chest_indices) == quicksum(x[i] for i in back_indices), 
    "Chest_Back"
)

# Constraint 5: Biceps vs Triceps equality
biceps_indices = workout[workout['BodyPart'] == "Biceps"].index
triceps_indices = workout[workout['BodyPart'] == "Triceps"].index
model.addConstr(
    quicksum(x[i] for i in biceps_indices) == quicksum(x[i] for i in triceps_indices),
    "Biceps_Triceps"
)

# Constraint 6: SFR ≤ 0.55
sfr = workout['Stimulus-to-Fatigue'].values
model.addConstr(quicksum(sfr[i] * x[i] for i in range(len(workout))) <= 0.55, name="SFR")

# Constraint 7: Beginner/Intermediate/Advanced ratios
beginner_indices = workout[workout['Difficulty'] == "Beginner"].index
intermediate_indices = workout[workout['Difficulty'] == "Intermediate"].index
expert_indices = workout[workout['Difficulty'] == "Expert"].index
model.addConstr(quicksum(x[i] for i in beginner_indices) >= 1.4 * quicksum(x[i] for i in intermediate_indices))
model.addConstr(
    quicksum(x[i] for i in intermediate_indices) >= 1.1 * quicksum(x[i] for i in expert_indices), 
    "Intermediate_Expert"
)

# Constraint 8: Category constraints (Strongman, Powerlifting, Olympic)
strongman_indices = workout[workout['Category'] == "Strongman"].index
powerlifting_indices = workout[workout['Category'] == "Powerlifting"].index
olympic_indices = workout[workout['Category'] == "Olympic Weightlifting"].index

model.addConstr(quicksum(x[i] for i in strongman_indices) <= 0.08 - 1e-5, "Strongman_Limit")  # <8%
model.addConstr(quicksum(x[i] for i in powerlifting_indices) >= 0.09 + 1e-5, "Powerlifting_Min")  # >9%
model.addConstr(quicksum(x[i] for i in olympic_indices) >= 0.10 + 1e-5, "Olympic_Min")  # >10%

# Constraint 9: Equipment usage ≥60%
equipment_list = ["Barbell", "Dumbbell", "Machine", "Cable", "E-Z Curl Bar", "Bands"]
equipment_indices = workout[workout['Equipment'].isin(equipment_list)].index
model.addConstr(quicksum(x[i] for i in equipment_indices) >= 0.6, "Equipment_Min")

# Optimize the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Hypertrophy Rating: {model.objVal}")
    for i in range(len(workout)):
        if x[i].x > 0:
            print(f"{workout['Exercise'][i]}: {x[i].x:.4f}")
else:
    print("Model is infeasible. Check dataset or constraints!")

# %%
# Validation checks (same as before)
# Check available Chest/Back exercises
chest_exercises = workout[workout['BodyPart'] == "Chest"]
back_exercises = workout[workout['BodyPart'] == "Back"]
print(f"Chest exercises: {len(chest_exercises)}")
print(f"Back exercises: {len(back_exercises)}")

# Check SFR values for Chest/Back exercises
print("Chest SFR stats:", chest_exercises['Stimulus-to-Fatigue'].describe())
print("Back SFR stats:", back_exercises['Stimulus-to-Fatigue'].describe())

# %%
# Check categories and constraints
print("Categories:", workout['Category'].unique())
print("BodyParts:", workout['BodyPart'].unique())

# %%
# Check model status and IIS if needed
if model.status == GRB.INFEASIBLE:
    print("Model is infeasible. Computing IIS...")
    model.computeIIS()
    for c in model.getConstrs():
        if c.IISConstr:
            print(f"Conflict in constraint: {c.ConstrName}")

# %%
# Post-optimization validation
if model.status == GRB.OPTIMAL:
    # Check total allocation
    total = sum(x[i].x for i in range(len(workout)))
    print(f"Total Allocation: {total:.6f}")  # Should be 1.0

    # Check SFR
    sfr_total = sum(workout['Stimulus-to-Fatigue'][i] * x[i].x for i in range(len(workout)))
    print(f"SFR Total: {sfr_total:.6f}")  # Should be ≤0.55

    # Check Biceps/Triceps equality
    biceps_total = sum(x[i].x for i in biceps_indices)
    triceps_total = sum(x[i].x for i in triceps_indices)
    print(f"Biceps: {biceps_total:.4f}, Triceps: {triceps_total:.4f}")

    # Check category constraints
    strongman_total = sum(x[i].x for i in strongman_indices)
    powerlifting_total = sum(x[i].x for i in powerlifting_indices)
    olympic_total = sum(x[i].x for i in olympic_indices)
    print(f"Strongman: {strongman_total:.4f} (≤8%), Powerlifting: {powerlifting_total:.4f} (≥9%), Olympic: {olympic_total:.4f} (≥10%)")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 26 rows, 2400 columns and 15976 nonzeros
Model fingerprint: 0x77888ef9
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [5e-02, 5e-02]
  RHS range        [5e-03, 1e+00]
Presolve time: 0.00s
Presolved: 26 rows, 2400 columns, 15976 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   2.755010e+00   0.000000e+00      0s
      28    7.8574257e-01   0.000000e+00   0.000000e+00      0s

Solved in 28 iterations and 0.02 seconds (0.01 work units)
Optimal objective  7.857425731e-01
Optimal Hypertrophy Rating: 0.7857425730587836
Clean Deadlift: 0.0500
Muscle Snatch: 0.0500
Clean Shrug: 0.0050
Clean from Blocks: 0.0415
Power sn

In [17]:
barbell_squat_index = workout[workout['Exercise'] == "Barbell Back Squats"].index[0]

In [18]:
reduced_cost = x[barbell_squat_index].RC
print(f"Required HR increase: {reduced_cost:.4f}")

Required HR increase: -0.4544


In [19]:
print(workout.loc[barbell_squat_index, 'Stimulus-to-Fatigue'])

0.648706348


# Part G

In [20]:
# Check SFR of random excluded common exercises
common_exercises = ["Bench Press", "Deadlift", "Barbell Back Squats"]
filtered = workout[workout['Exercise'].isin(common_exercises)]
print(filtered[['Exercise', 'Stimulus-to-Fatigue', 'Hypertrophy Rating']])

                Exercise  Stimulus-to-Fatigue  Hypertrophy Rating
293  Barbell Back Squats             0.648706            0.678346


# Part H

In [21]:


# %%
# --------------------------
# Reclassify Back Subparts
# --------------------------
back_subparts = ["Lats", "Lower Back", "Middle Back"]  # Adjust based on your dataset
workout.loc[workout['BodyPart'].isin(back_subparts), 'BodyPart'] = "Back"

# --------------------------
# Filter High-SFR Chest Exercises
# --------------------------
workout = workout[
    ~((workout['BodyPart'] == "Chest") & (workout['Stimulus-to-Fatigue'] > 0.55))
]

# Reset indices to avoid KeyError
workout = workout.reset_index(drop=True)

# --------------------------
# Define the Model and Constraints
# --------------------------
model = Model("Workout_Optimization")
x = model.addVars(len(workout), lb=0, ub=0.05, vtype=GRB.CONTINUOUS, name="x")

# Objective: Maximize hypertrophy rating
hypertrophy = workout['Hypertrophy Rating'].values
model.setObjective(quicksum(hypertrophy[i] * x[i] for i in range(len(workout))), GRB.MAXIMIZE)

# Constraint 1: Total allocation = 1
model.addConstr(quicksum(x[i] for i in range(len(workout))) == 1, "Total_Allocation")

# Constraint 2: Body part minimums
body_parts_min = {
    "Traps": 0.005, "Neck": 0.005, "Forearms": 0.005, "Abdominals": 0.04
}
all_body_parts = workout['BodyPart'].unique()
for body_part in all_body_parts:
    exercises = workout[workout['BodyPart'] == body_part].index
    if body_part in body_parts_min:
        min_val = body_parts_min[body_part]
    else:
        min_val = 0.025  # 2.5% default
    model.addConstr(quicksum(x[i] for i in exercises) >= min_val, f"Min_{body_part}")



# Constraint 9: Equipment usage ≥60%
equipment_list = ["Barbell", "Dumbbell", "Machine", "Cable", "E-Z Curl Bar", "Bands"]
equipment_indices = workout[workout['Equipment'].isin(equipment_list)].index
model.addConstr(quicksum(x[i] for i in equipment_indices) >= 0.6, "Equipment_Min")

# Optimize the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Hypertrophy Rating: {model.objVal}")
    for i in range(len(workout)):
        if x[i].x > 0:
            print(f"{workout['Exercise'][i]}: {x[i].x:.4f}")
else:
    print("Model is infeasible. Check dataset or constraints!")

# %%
# Validation checks (same as before)
# Check available Chest/Back exercises
chest_exercises = workout[workout['BodyPart'] == "Chest"]
back_exercises = workout[workout['BodyPart'] == "Back"]
print(f"Chest exercises: {len(chest_exercises)}")
print(f"Back exercises: {len(back_exercises)}")

# Check SFR values for Chest/Back exercises
print("Chest SFR stats:", chest_exercises['Stimulus-to-Fatigue'].describe())
print("Back SFR stats:", back_exercises['Stimulus-to-Fatigue'].describe())

# %%
# Check categories and constraints
print("Categories:", workout['Category'].unique())
print("BodyParts:", workout['BodyPart'].unique())

# %%
# Check model status and IIS if needed
if model.status == GRB.INFEASIBLE:
    print("Model is infeasible. Computing IIS...")
    model.computeIIS()
    for c in model.getConstrs():
        if c.IISConstr:
            print(f"Conflict in constraint: {c.ConstrName}")

# %%
# Post-optimization validation
if model.status == GRB.OPTIMAL:
    # Check total allocation
    total = sum(x[i].x for i in range(len(workout)))
    print(f"Total Allocation: {total:.6f}")  # Should be 1.0

    # Check SFR
    sfr_total = sum(workout['Stimulus-to-Fatigue'][i] * x[i].x for i in range(len(workout)))
    print(f"SFR Total: {sfr_total:.6f}")  # Should be ≤0.55

    # Check Biceps/Triceps equality
    biceps_total = sum(x[i].x for i in biceps_indices)
    triceps_total = sum(x[i].x for i in triceps_indices)
    print(f"Biceps: {biceps_total:.4f}, Triceps: {triceps_total:.4f}")

    # Check category constraints
    strongman_total = sum(x[i].x for i in strongman_indices)
    powerlifting_total = sum(x[i].x for i in powerlifting_indices)
    olympic_total = sum(x[i].x for i in olympic_indices)
    print(f"Strongman: {strongman_total:.4f} (≤8%), Powerlifting: {powerlifting_total:.4f} (≥9%), Olympic: {olympic_total:.4f} (≥10%)")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 17 rows, 2400 columns and 5969 nonzeros
Model fingerprint: 0xf0082319
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [5e-02, 5e-02]
  RHS range        [5e-03, 1e+00]
Presolve time: 0.03s
Presolved: 17 rows, 2400 columns, 5969 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   1.255000e+00   0.000000e+00      0s
      12    8.6155916e-01   0.000000e+00   0.000000e+00      0s

Solved in 12 iterations and 0.05 seconds (0.00 work units)
Optimal objective  8.615591622e-01
Optimal Hypertrophy Rating: 0.8615591621749997
Behind-the-head push-press: 0.0500
Board bench press: 0.0500
Barbell glute bridge: 0.0500
Barbell Hip

# Part I

In [27]:
# Prepare data for modeling
n = len(workout)  # Total number of exercises
HR = workout['Hypertrophy Rating'].tolist()  # Hypertrophy ratings
body_parts = workout['BodyPart'].tolist()  # Body part categories
equipment = workout['Equipment'].fillna("Unknown").tolist()  
exercises = workout['Exercise'].tolist()  

# Define body part categories for constraints
body_part_min_alloc = {
    'Traps': 0.005,
    'Neck': 0.005,
    'Forearms': 0.005,
    'Abdominals': 0.04
}
default_min_alloc = 0.025
# Identify all unique body parts
all_body_parts = set(body_parts)
other_bps = all_body_parts - set(body_part_min_alloc.keys())

# Identify key equipment indices
key_equipment = ['Barbell', 'Dumbbell', 'Machine', 'Cable', 'E-Z Curl Bar', 'Bands']
key_equipment_indices = [i for i in range(n) if equipment[i] in key_equipment]

# --------------------------
# Initialize primal Model 
# --------------------------
primal_model = Model("Primal_Optimized_Workout")

# %%
# Define Decision Variables
x = primal_model.addVars(n, vtype=GRB.CONTINUOUS, lb=0, name="x")

# %%
# Set Objective: Maximize sum(HR_i * x_i)
primal_model.setObjective(quicksum(HR[i] * x[i] for i in range(n)), GRB.MAXIMIZE)

primal_model.addConstr(quicksum(x[i] for i in range(n)) == 1, "TotalProportion")

# %%
# Add Constraint: No Single Exercise Exceeds 5%
primal_model.addConstrs((x[i] <= 0.05 for i in range(n)), "MaxExercise")

# %%
# Add Constraint: Minimum Allocation per Body Part
primal_model.addConstrs(
    (quicksum(x[i] for i in range(n) if body_parts[i] == b) >= body_part_min_alloc[b]
     for b in body_part_min_alloc.keys()),
    "MinAlloc_Specific"
)
primal_model.addConstrs(
    (quicksum(x[i] for i in range(n) if body_parts[i] == b) >= default_min_alloc
     for b in other_bps),
    "MinAlloc_Default"
)

# %%
# Add Constraint: Equipment Usage Constraint
primal_model.addConstr(
    quicksum(x[i] for i in key_equipment_indices) >= 0.6,
    "EquipmentUsage"
)

# %%
# Disable Presolve to maintain all constraints and variables
primal_model.setParam('Presolve', 0)

# %%
# Optimize the primal model
primal_model.optimize()

# %%
# Check if the primal model has an optimal solution
if primal_model.status == GRB.OPTIMAL:
    primal_optimal_hypertrophy = primal_model.objVal
    print(f"Primal Optimal Hypertrophy Rating: {primal_optimal_hypertrophy:.6f}")

    # Retrieve and display exercise allocations
    solution = {exercises[i]: x[i].X for i in range(n) if x[i].X > 1e-6}
    print("\nExercise Allocations:")
    for exercise, allocation in solution.items():
        print(f"{exercise}: {allocation:.4f}")

    # Retrieve Dual Variables (Shadow Prices)
    dual_y = primal_model.getConstrByName("TotalProportion").Pi
    dual_z = {i: primal_model.getConstrByName(f"MaxExercise[{i}]").Pi for i in range(n)}
    dual_w = {
        b: primal_model.getConstrByName(f"MinAlloc_Specific[{b}]").Pi
        for b in body_part_min_alloc.keys()
    }
    dual_w.update({
        b: primal_model.getConstrByName(f"MinAlloc_Default[{b}]").Pi
        for b in other_bps
    })
    dual_v = primal_model.getConstrByName("EquipmentUsage").Pi

    # %%
    # Calculate Dual Objective
    dual_objective = (
        dual_y
        + sum(0.05 * dual_z[i] for i in range(n))
        + sum(body_part_min_alloc[b] * dual_w[b] for b in body_part_min_alloc.keys())
        + sum(default_min_alloc * dual_w[b] for b in other_bps)
        + 0.6 * dual_v
    )

    print(f"\nDual Optimal Objective (Calculated): {dual_objective:.6f}")

    # %%
    # Display Dual Variables
    print("\nDual Variable Values:")
    print(f"Dual Variable y (Total Proportion): {dual_y:.6f}")
    print("\nDual Variables z_i (Max Single Exercise Constraints):")
    for i in range(n):
        if dual_z[i] > 1e-6:
            print(f"z_{i} (Exercise '{exercises[i]}'): {dual_z[i]:.6f}")
    print("\nDual Variables w_b (Min Body Part Allocations):")
    for b in dual_w:
        print(f"w_{b}: {dual_w[b]:.6f}")
    print(f"\nv (Equipment Usage): {dual_v:.6f}")

    # %%
    # Compare Primal and Dual Objectives
    print(f"\nPrimal Optimal Hypertrophy Rating: {primal_optimal_hypertrophy:.6f}")
    print(f"Dual Optimal Objective (Calculated): {dual_objective:.6f}")
    print(f"Dual Objective >= Primal Objective: {dual_objective >= primal_optimal_hypertrophy}")

    if abs(dual_objective - primal_optimal_hypertrophy) <= 1e-6:
        print("Strong Duality holds: Dual Objective equals Primal Objective.")
    else:
        print("Duality Gap exists: Dual Objective does not equal Primal Objective.")
else:
    print("No optimal solution found for the primal model.")

Set parameter Presolve to value 0
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1235U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
Presolve  0

Optimize a model with 2417 rows, 2400 columns and 8369 nonzeros
Model fingerprint: 0x43aa6fb1
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3432472e+33   4.800000e+33   1.343247e+03      0s
    3704    8.6155916e-01   0.000000e+00   0.000000e+00      0s

Solved in 3704 iterations and 0.10 seconds (0.10 work units)
Optimal objective  8.615591622e-01
Primal Optimal Hypertrophy Rating: 0.861559

Exercise Allocations:
Behind-the-head push-press: 0.0500
Board bench press: 0.0500
Barbell glute 